## Importing relevant libraries

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Importing dataset

In [2]:
mnist, mnist_info = tfds.load('mnist', as_supervised = True, with_info = True)

In [3]:
mnist_train, mnist_test = mnist['train'], mnist['test']

## Defining number of valitation and test samples

In [4]:
num_valid_samples = 0.1 * mnist_info.splits['train'].num_examples
num_valid_samples = tf.cast(num_valid_samples, tf.int64)

In [5]:
num_test_samples = 0.1 * mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

## Scaling the data

In [6]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return (image, label)

In [7]:
scaled_train_and_valid = mnist_train.map(scale)

In [8]:
scaled_test = mnist_test.map(scale)

## Shuffeling the data

In [9]:
BUFFER_SIZE = 15000

shuffle_train_test = scaled_train_and_valid.shuffle(BUFFER_SIZE)

In [10]:
valid_data = shuffle_train_test.take(num_valid_samples)
train_data = shuffle_train_test.skip(num_valid_samples)

## Batching the data

In [11]:
BATCH_SIZE = 100

In [12]:
train_data = train_data.batch(BATCH_SIZE)
valid_data = valid_data.batch(num_valid_samples)
test_data = scaled_test.batch(num_test_samples)

In [13]:
validation_inputs, validation_targets = next(iter(valid_data))

## Outlining the model

In [14]:
input_size = 784
output_size = 10
hidden_layer_size = 100



model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape = (28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(output_size, 'softmax')
                            ])

In [15]:
model.compile(optimizer = 'rmsprop', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

## Training the model and validation test accuracy

In [16]:
NUM_EPOCHS = 6


model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets),validation_steps = 1, verbose = 2)

Epoch 1/6
540/540 - 18s - loss: 0.3172 - accuracy: 0.9052 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/6
540/540 - 13s - loss: 0.1302 - accuracy: 0.9604 - val_loss: 0.1175 - val_accuracy: 0.9662
Epoch 3/6
540/540 - 13s - loss: 0.0912 - accuracy: 0.9719 - val_loss: 0.0855 - val_accuracy: 0.9747
Epoch 4/6
540/540 - 13s - loss: 0.0726 - accuracy: 0.9773 - val_loss: 0.0848 - val_accuracy: 0.9755
Epoch 5/6
540/540 - 13s - loss: 0.0593 - accuracy: 0.9815 - val_loss: 0.0652 - val_accuracy: 0.9817
Epoch 6/6
540/540 - 13s - loss: 0.0479 - accuracy: 0.9854 - val_loss: 0.0507 - val_accuracy: 0.9852


## Testing the model

In [17]:
test_loss, test_accuracy = model.evaluate(test_data)

10/10 [==============================] - 3s 253ms/step - loss: 0.0879 - accuracy: 0.9755
